# AzionVectorStore

`AzionVectorStore` 用于在 Azion 的 Edge 平台上，使用 Edge SQL 直接管理和搜索文档集合中的向量嵌入。

本指南提供了关于如何开始使用 Azion EdgeSQL [向量存储](/docs/concepts/#vectorstores) 的快速概述。如需了解 `AzionVectorStore` 所有功能和配置的详细文档，请参阅 [API 参考文档](https://api.js.langchain.com/classes/_langchain_community.vectorstores_azion_edgesql.AzionVectorStore.html)。

## 概述

### 集成详情

| 类 | 包 | [PY 支持] |  包最新版本 |
| :--- | :--- | :---: | :---: |
| [`AzionVectorStore`](https://api.js.langchain.com/classes/langchain_community_vectorstores_azion_edgesql.AzionVectorStore.html) | [`@langchain/community`](https://npmjs.com/@langchain/community) | ❌ |  ![NPM - 版本](https://img.shields.io/npm/v/@langchain/community?style=flat-square&label=%20&) |

## 配置

要使用 `AzionVectorStore` 向量存储，你需要安装 `@langchain/community` 包。除此之外，你还需要一个 [Azion 账户](https://www.azion.com/en/documentation/products/accounts/creating-account/) 和一个 [Token](https://www.azion.com/en/documentation/products/guides/personal-tokens/) 来使用 Azion API，并将其配置为环境变量 `AZION_TOKEN`。更多相关信息可以在 [文档](https://www.azion.com/en/documentation/) 中找到。

本指南还将使用 [OpenAI 嵌入](/docs/integrations/text_embedding/openai)，这需要你安装 `@langchain/openai` 集成包。如果需要，你也可以使用 [其他支持的嵌入模型](/docs/integrations/text_embedding)。

```{=mdx}
import IntegrationInstallTooltip from "@mdx_components/integration_install_tooltip.mdx";
import Npm2Yarn from "@theme/Npm2Yarn";

<IntegrationInstallTooltip></IntegrationInstallTooltip>

<Npm2Yarn>
  azion @langchain/openai @langchain/community
</Npm2Yarn>
```

### 凭证

完成这些步骤后，请设置 AZION_TOKEN 环境变量：

```typescript
process.env.AZION_TOKEN = "your-api-key"
```

如果你在本指南中使用 OpenAI 嵌入，还需要设置你的 OpenAI 密钥：

```typescript
process.env.OPENAI_API_KEY = "YOUR_API_KEY";
```

如果你想获得模型调用的自动追踪，也可以通过取消注释以下内容来设置你的 [LangSmith](https://docs.smith.langchain.com/) API 密钥：

```typescript
// process.env.LANGCHAIN_TRACING_V2="true"
// process.env.LANGCHAIN_API_KEY="your-api-key"
```

## 实例化

In [64]:
import { AzionVectorStore } from "@langchain/community/vectorstores/azion_edgesql";
import { OpenAIEmbeddings } from "@langchain/openai";

const embeddings = new OpenAIEmbeddings({
  model: "text-embedding-3-small",
});

// Instantiate with the constructor if the database and table have already been created
const vectorStore = new AzionVectorStore(embeddings, { dbName: "langchain", tableName: "documents" });

// If you have not created the database and table yet, you can do so with the setupDatabase method
// await vectorStore.setupDatabase({ columns:["topic","language"], mode: "hybrid" })

// OR instantiate with the static method if the database and table have not been created yet
// const vectorStore = await AzionVectorStore.initialize(embeddingModel, { dbName: "langchain", tableName: "documents" }, { columns:[], mode: "hybrid" })

## 管理向量存储

### 向向量存储中添加项目

In [52]:
import type { Document } from "@langchain/core/documents";

const document1: Document = {
  pageContent: "The powerhouse of the cell is the mitochondria",
  metadata: { language: "en", topic: "biology" }
};

const document2: Document = {
  pageContent: "Buildings are made out of brick",
  metadata: { language: "en", topic: "history" }
};

const document3: Document = {
  pageContent: "Mitochondria are made out of lipids",
  metadata: { language: "en", topic: "biology" }
};

const document4: Document = {
  pageContent: "The 2024 Olympics are in Paris",
  metadata: { language: "en", topic: "history" }
}

const documents = [document1, document2, document3, document4];

await vectorStore.addDocuments(documents);

Inserting chunks
Inserting chunk 0
Chunks inserted!


### 从向量存储中删除项目

In [53]:
await vectorStore.delete(["4"]);

Deleted 1 items from documents


## 查询向量存储

一旦创建了向量存储并添加了相关文档，在运行链或代理时很可能需要对其进行查询。

### 直接查询

执行一个简单的相似性搜索可以按以下方式进行：

In [66]:
const filter = [{ operator: "=", column: "language", value: "en" }]

const hybridSearchResults = await vectorStore.azionHybridSearch("biology", {kfts:2, kvector:1, 
                                      filter:[{ operator: "=", column: "language", value: "en" }]});

console.log("Hybrid Search Results")
for (const doc of hybridSearchResults) {
  console.log(`${JSON.stringify(doc)}`);
}

Hybrid Search Results
[{"pageContent":"The Australian dingo is a unique species that plays a key role in the ecosystem","metadata":{"searchtype":"fulltextsearch"},"id":"6"},-0.25748711028997995]
[{"pageContent":"The powerhouse of the cell is the mitochondria","metadata":{"searchtype":"fulltextsearch"},"id":"16"},-0.31697985337654005]
[{"pageContent":"Australia s indigenous people have inhabited the continent for over 65,000 years","metadata":{"searchtype":"similarity"},"id":"3"},0.14822345972061157]


In [67]:
const similaritySearchResults = await vectorStore.azionSimilaritySearch("australia", {kvector:3, filter:[{ operator: "=", column: "topic", value: "history" }]});

console.log("Similarity Search Results")
for (const doc of similaritySearchResults) {
  console.log(`${JSON.stringify(doc)}`);
}

Similarity Search Results
[{"pageContent":"Australia s indigenous people have inhabited the continent for over 65,000 years","metadata":{"searchtype":"similarity"},"id":"3"},0.4486490488052368]


### 通过转换为检索器进行查询

你还可以将向量存储转换为[检索器](/docs/concepts/#retrievers)，以便在你的链中更方便地使用。

In [56]:
const retriever = vectorStore.asRetriever({
  // Optional filter
  filter: filter,
  k: 2,
});
await retriever.invoke("biology");

[
  Document {
    pageContent: 'Australia s indigenous people have inhabited the continent for over 65,000 years',
    metadata: { searchtype: 'similarity' },
    id: '3'
  },
  Document {
    pageContent: 'Mitochondria are made out of lipids',
    metadata: { searchtype: 'similarity' },
    id: '18'
  }
]


### 检索增强生成的用法

有关如何将此向量存储用于检索增强生成（RAG）的指南，请参阅以下部分：

- [教程：使用外部知识](/docs/tutorials/#working-with-external-knowledge)。
- [如何操作：使用RAG进行问答](/docs/how_to/#qa-with-rag)
- [检索概念文档](/docs/concepts/retrieval)

## API 参考文档

如需详细了解 AzionVectorStore 的所有功能和配置，请访问 [API 参考文档](https://api.js.langchain.com/classes/_langchain_community.vectorstores_azion_edgesql.AzionVectorStore.html)。